# Advanced Convergence Analysis

## Overview
Deep-dive into Monte Carlo convergence diagnostics: running-mean stability, effective sample size (ESS), MCMC-style chain mixing, batch-means standard errors, and practical guidance on choosing simulation counts.

- **Prerequisites**: [core/03_monte_carlo_simulation](../core/03_monte_carlo_simulation.ipynb)
- **Estimated runtime**: 1-2 minutes
- **Audience**: [Developer]

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance import ManufacturerConfig, InsuranceProgram, EnhancedInsuranceLayer
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.monte_carlo import MonteCarloEngine

plt.style.use("seaborn-v0_8-darkgrid")

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Baseline Simulation

Generate a large Monte Carlo run that serves as our "ground truth" for convergence analysis.

In [ ]:
config = ManufacturerConfig(
    initial_assets=10_000_000,
    asset_turnover_ratio=1.0,
    base_operating_margin=0.10,
    tax_rate=0.25,
    retention_ratio=0.7,
)

loss_gen = ManufacturingLossGenerator(
    attritional_params={"base_frequency": 4.0, "severity_mean": 30_000, "severity_cv": 0.6},
    large_params={"base_frequency": 0.4, "severity_mean": 400_000, "severity_cv": 0.8},
    catastrophic_params={"base_frequency": 0.02, "severity_xm": 3_000_000, "severity_alpha": 2.0},
    seed=SEED,
)

program = InsuranceProgram([
    EnhancedInsuranceLayer(attachment_point=50_000, limit=2_000_000,
                           base_premium_rate=0.015),
    EnhancedInsuranceLayer(attachment_point=2_050_000, limit=5_000_000,
                           base_premium_rate=0.005),
])
premium = float(program.calculate_annual_premium())

N_SIMS = 5_000
N_YEARS = 10

final_assets = np.zeros(N_SIMS)
for s in range(N_SIMS):
    m = WidgetManufacturer(config)
    for yr in range(N_YEARS):
        _, st = loss_gen.generate_losses(1.0, float(m.calculate_revenue()))
        rec = program.process_claim(st["total_amount"])
        net = st["total_amount"] - rec["insurance_recovery"]
        if net > 0:
            m.process_insurance_claim(net)
        if premium > 0:
            m.record_insurance_premium(premium)
        m.step(growth_rate=0.0)
        if float(m.equity) <= 0:
            break
    final_assets[s] = max(float(m.total_assets), 0)

surviving = final_assets[final_assets > 0]
growth_rates = (surviving / config.initial_assets) ** (1 / N_YEARS) - 1

print(f"Simulations : {N_SIMS}")
print(f"Survivors   : {len(surviving)} ({len(surviving)/N_SIMS:.1%})")
print(f"Ruin rate   : {1 - len(surviving)/N_SIMS:.2%}")
print(f"Mean CAGR   : {np.mean(growth_rates):.3%}")
print(f"Median CAGR : {np.median(growth_rates):.3%}")

## 2. Running-Mean Convergence

Plot the running (cumulative) mean of the growth rate as more simulations are added.
A stable running mean indicates convergence.

In [ ]:
running_mean = np.cumsum(growth_rates) / np.arange(1, len(growth_rates) + 1)
running_std  = np.array([growth_rates[:i+1].std() for i in range(len(growth_rates))])
se = running_std / np.sqrt(np.arange(1, len(growth_rates) + 1))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
n = np.arange(1, len(running_mean) + 1)
ax.plot(n, running_mean * 100, lw=1.5, label="Running mean")
ax.fill_between(n, (running_mean - 2 * se) * 100, (running_mean + 2 * se) * 100,
                alpha=0.2, label="95% CI")
ax.axhline(np.mean(growth_rates) * 100, ls="--", color="red", alpha=0.5,
           label=f"Final mean={np.mean(growth_rates):.2%}")
ax.set_xlabel("Number of Simulations")
ax.set_ylabel("Mean CAGR (%)")
ax.set_title("Running Mean Convergence")
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(n, se * 100, lw=1.5, color="orange")
ax.set_xlabel("Number of Simulations")
ax.set_ylabel("Standard Error (%)")
ax.set_title("Standard Error Decay")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Report convergence at key sample sizes
for k in [100, 500, 1000, 2000, len(growth_rates)]:
    idx = min(k, len(growth_rates)) - 1
    print(f"  n={k:>5d}  mean={running_mean[idx]:.3%}  SE={se[idx]:.4%}")

## 3. Effective Sample Size (ESS)

Autocorrelation reduces the effective number of independent samples.
We estimate ESS using the initial monotone sequence estimator.

In [ ]:
def compute_ess(x):
    """Effective sample size via initial positive sequence estimator."""
    n = len(x)
    x_centered = x - x.mean()
    acf = np.correlate(x_centered, x_centered, mode="full")[n - 1:]
    acf /= acf[0]
    # sum positive pairs
    tau = 1.0
    for k in range(1, n // 2):
        pair_sum = acf[2 * k - 1] + acf[2 * k]
        if pair_sum < 0:
            break
        tau += 2 * pair_sum
    return n / tau

ess = compute_ess(growth_rates)
ess_ratio = ess / len(growth_rates)

print(f"Nominal samples : {len(growth_rates)}")
print(f"Effective samples: {ess:.0f}  ({ess_ratio:.1%} efficiency)")

# Autocorrelation plot
max_lag = 50
acf_vals = [np.corrcoef(growth_rates[:-lag], growth_rates[lag:])[0, 1]
            for lag in range(1, max_lag + 1)]

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(range(1, max_lag + 1), acf_vals, color="steelblue", alpha=0.7)
ax.axhline(0, color="black", lw=0.5)
ax.axhline( 2 / np.sqrt(len(growth_rates)), ls="--", color="red", alpha=0.5)
ax.axhline(-2 / np.sqrt(len(growth_rates)), ls="--", color="red", alpha=0.5)
ax.set_xlabel("Lag")
ax.set_ylabel("Autocorrelation")
ax.set_title("Sample Autocorrelation of Growth Rates")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Batch-Means Standard Error

Divide the sample into batches and use the between-batch variance to estimate the standard error.
This is robust to serial correlation.

In [ ]:
def batch_means_se(x, n_batches=20):
    n = len(x)
    batch_size = n // n_batches
    batch_means = np.array([x[i * batch_size:(i + 1) * batch_size].mean()
                            for i in range(n_batches)])
    return batch_means.std() / np.sqrt(n_batches), batch_means

bm_se, batch_vals = batch_means_se(growth_rates)
naive_se = growth_rates.std() / np.sqrt(len(growth_rates))

print(f"Naive SE        : {naive_se:.5%}")
print(f"Batch-means SE  : {bm_se:.5%}")
print(f"Ratio (BM/Naive): {bm_se / naive_se:.2f}")

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(range(len(batch_vals)), batch_vals * 100, color="teal", alpha=0.7)
ax.axhline(np.mean(growth_rates) * 100, ls="--", color="red",
           label=f"Overall mean={np.mean(growth_rates):.2%}")
ax.set_xlabel("Batch")
ax.set_ylabel("Batch Mean CAGR (%)")
ax.set_title("Batch Means")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Convergence by Metric

Different quantities converge at different rates.  Compare ruin probability, mean growth, and VaR.

In [ ]:
sample_sizes = np.arange(50, len(final_assets) + 1, 50)
ruin_conv, mean_conv, var95_conv = [], [], []

for n in sample_sizes:
    fa = final_assets[:n]
    surv = fa[fa > 0]
    ruin_conv.append(1 - len(surv) / n)
    if len(surv) > 1:
        gr = (surv / config.initial_assets) ** (1 / N_YEARS) - 1
        mean_conv.append(np.mean(gr))
        var95_conv.append(np.percentile(gr, 5))
    else:
        mean_conv.append(np.nan)
        var95_conv.append(np.nan)

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
for ax, vals, title in zip(axes,
    [ruin_conv, mean_conv, var95_conv],
    ["Ruin Probability", "Mean CAGR", "5th Percentile CAGR (VaR)"],
):
    ax.plot(sample_sizes, vals, lw=1.5)
    if not np.all(np.isnan(vals)):
        ax.axhline(vals[-1], ls="--", color="red", alpha=0.5)
    ax.set_xlabel("N simulations")
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.suptitle("Convergence by Metric", fontweight="bold")
plt.tight_layout()
plt.show()

print("Tail metrics (ruin, VaR) require more simulations than central metrics (mean).")

## 6. Practical Guidance: How Many Simulations?

Use the standard-error budget to choose simulation count for a target precision.

In [ ]:
target_se = 0.002  # target SE on CAGR
sigma = growth_rates.std() if len(growth_rates) > 1 else 0.0

if sigma > 0 and np.isfinite(sigma):
    n_needed_naive = int(np.ceil((sigma / target_se) ** 2))
    n_needed_ess   = int(np.ceil(n_needed_naive / ess_ratio)) if ess_ratio > 0 else n_needed_naive
else:
    n_needed_naive = 0
    n_needed_ess = 0

print(f"Sample std of CAGR: {sigma:.4%}")
print(f"Target SE          : {target_se:.4%}")
print(f"Needed (iid)       : {n_needed_naive:,}")
print(f"Needed (ESS-adj)   : {n_needed_ess:,}")

p_hat = 1 - len(surviving) / N_SIMS
print(f"\nFor ruin probability estimation at {p_hat:.2%}:")
if p_hat > 0:
    n_ruin = int(np.ceil(p_hat * (1 - p_hat) / (0.005 ** 2)))  # SE=0.5%
    print(f"  Needed for SE < 0.5%: {n_ruin:,}")
else:
    print(f"  No ruin events -- need many more simulations to estimate.")

## Key Takeaways

- **Running-mean plots** are the simplest convergence diagnostic -- look for a flat tail.
- **ESS** quantifies how much autocorrelation inflates uncertainty; low ESS means results are less reliable than the nominal sample count suggests.
- **Batch-means SE** is robust to serial correlation and should be preferred over naive SE.
- **Tail metrics** (ruin probability, VaR) converge much more slowly than central moments; budget 5-10x more simulations for tails.
- Use the SE-budget formula to determine the simulation count **before** running expensive optimisations.

## Next Steps

- [core/03_monte_carlo_simulation](../core/03_monte_carlo_simulation.ipynb) -- Monte Carlo engine basics
- [advanced/02_walk_forward_validation](02_walk_forward_validation.ipynb) -- out-of-sample strategy validation
- [optimization/01_optimization_overview](../optimization/01_optimization_overview.ipynb) -- optimization algorithms